In [1]:
# 리스트 형태로 답변 반환
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from dotenv import load_dotenv
import torch
import sys
import os
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline

os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
load_dotenv()

True

In [2]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

In [3]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [4]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
quantization_config = BitsAndBytesConfig(load_in_8bit=True,llm_int8_threshold=6.0)          # 8bit 양자화 설정
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=quantization_config,  # 8bit 양자화
    device_map="cuda:0"  # 명시적 GPU 설정
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=40,
    temperature=0.01,  # 거의 0에 가까운 값 (일관성 유지)
    do_sample=True,    # 샘플링 활성화
    return_full_text=False,
    # device=0,  # device_map="cuda:0" 사용 시 제거
    pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 명시적 설정
)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23432\711731897.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [5]:
chain = prompt | llm | output_parser

In [6]:
answer = chain.invoke({"subject":"대한민국 관광명소, 한국어로 대답"})

In [7]:
answer

['', '1. 경상도 관광명소', '2. 경주 관광명소', '3. 전라남도 관광명소', '4. 전라북도 관광명소']